# Using the EFD python client to query metrics from analysis_tools

**Description:** Demonstrate how to query metric values produced with [analysis_tools](https://github.com/lsst/analysis_tools)[analysis_tools](https://github.com/lsst/analysis_tools) using the EFD python client for the Sasquatch environment. 

More information at https://sasquatch.lsst.io/

**Contact authors:** Keith Bechtol

**Last verified to run:** 2023-09-13

**LSST Science Piplines version:** w_2023_34

**Container Size:** any

**Location:** This notebook was developed on the S3DF cluster at the USDF.

In [ ]:
!eups list -s | grep lsst_distrib

Currently, the analysis_tools metrics are dispatched to the USDF dev Sasquatch environment, indicated with the `usdfdev_efd` alias, and are located in the `lsst.dm` database.

In [ ]:
from lsst_efd_client import EfdClient
client = EfdClient("usdfdev_efd", db_name="lsst.dm")

The topics correspond to individual analysis tools, e.g., as specified in the pipeline yaml file [coaddQualityCore.yaml](https://github.com/lsst/analysis_tools/blob/18877759150ddbd865aecc4be00199d1975baee4/pipelines/coaddQualityCore.yaml#L9). Consider `e1Diff` as an example.

In [ ]:
topics = await client.get_topics()
for t in topics: print(t)

For a given topic, the fields correspond to metadata and metrics. For example, the metrics in the topic `e1Diff` where specified in the [E1Diff](https://github.com/lsst/analysis_tools/blob/18877759150ddbd865aecc4be00199d1975baee4/python/lsst/analysis/tools/atools/shapes.py#L141) analysis tool class. 

In [ ]:
await client.get_fields("lsst.dm.e1Diff")

We can query a given topic using [InfluxQL syntax](https://docs.influxdata.com/influxdb/v1/query_language/explore-data). Notice that the columns in the returned DataFrame match the fields above.

In [ ]:
query = '''SELECT * FROM "lsst.dm.e1Diff" WHERE time > now() - 100d '''
df = await client.influx_client.query(query)
df.columns.values

Inspect the table. Each row corresponds to one `MetricMeasurementBundle` from analysis tools.

In [ ]:
df

Suppose that we are only interested in metrics with a given tag, e.g., `HSC/RC2`. We can refine our query as follows to select on field values.

In [ ]:
query = '''SELECT * FROM "lsst.dm.e1Diff" WHERE time > now() - 100d AND "dataset_tag" = 'HSC/RC2' '''
df = await client.influx_client.query(query)
df

Finally, access the specific metric values.

In [ ]:
df.g_highSNStars_count.values